SQL
===

**Juan David Velásquez Henao**  
jdvelasq@unal.edu.co   
Universidad Nacional de Colombia, Sede Medellín  
Facultad de Minas  
Medellín, Colombia


---

Haga click [aquí](https://github.com/jdvelasq/bash-for-analytics/tree/master/) para acceder al repositorio online.

Haga click [aquí](http://nbviewer.jupyter.org/github/jdvelasq/bash-for-analytics/tree/master/) para explorar el repositorio usando `nbviewer`. 

---

# `csvkit`

`csvkit` es una libreria de Python que permite ejecutar consultas SQL sobre archivos en formato CSV. Este lenguaje es utilizado en los motores de bases de datos para acceder a la información de manera práctica y resumida. Se abordará unicamente con efectos de consulta, no para administración de sistemas gestores de bases de datos.

Esta herramienta se instala a través del siguiente comando directamente en Python o en el Anaconda Promt:

    pip install csvkit
    

## `in2csv`

Los archivos a utilizar serán los incluidos en la carpeta de ejemplo del curso, sin embargo, `csvkit` permite a través de la función `in2csv` transformar una fuente de datos en Excel a csv de acuerdo con la siguiente estructura:

       in2csv Archivo.xls > Data.csv

Haga click [aquí](http://csvkit.readthedocs.io/en/1.0.2/scripts/in2csv.html) para acceder a la documentación de la función.

In [1]:
cd file

## `csvcut`

`csvcut` es una función que permite gestionar por columnas; filtrar y editar los campos.

La opción `-n` imprime los nombres de las columnas y sus respectivos indices.

In [2]:
csvcut -n employee

  1: num
  2: employee.id
  3: fullname
  4: department
  5: ssn
  6: phone
  7: city
  8: maritalstatus
  9: birthdate


La opción `-c` permite filtrar columnas de interés y guardarlas en un nuevo archivo.

In [3]:
csvcut -c maritalstatus,city employee > new.csv
head new.csv

maritalstatus,city
Common-Law,ONEONTA
Married,PALOS VERDES ESTATES
Common-Law,WARSAW
Single,NEW CITY
Common-Law,CIALES
Single,BRODNAX
Single,ALEX CITY
Single,CUTLER
Single,KERSEY


# `csvsql`

`csvsql` es la función que permite ejercutar comandos SQL sobre los archivos, generando nuevos conjuntos de datos o incluyendo información directamente a la base de datos.

> Un tutorial de SQL es presentado [aquí](https://github.com/jdvelasq/SQL-basics).

El parámetro `--query` del comando `csvsql` permite ejecutar cualquier consulta del tipo DML (se usan para consultar, filtrar y ordenar datos) sobre un archivo usando el lenguaje SQL. Los elementos escenciales para este tipo de consulta son:

    SELECT             <campos o columnas en la tabla a visualizar> 
    SELECT DISTINCT    <valores únicos en los campos>
    FROM               <tabla o consulta> 
    WHERE              <condiciones sobre las columnas de la tabla> 
    AND/OR             <comando para indicar unión o intersección de condiciones>
    AS                 <renombrar una columna o nuevo campo>
    GROUP BY           <campos por los cuales se formaran conjuntos> 
    ORDER BY           <ordernar por un conjunto de columnas>
    LIMIT              <límite de filas que se mostratrán>


A continuación sobre el archivo `employee` se requiere guardar los datos de la columna 'fullname' y 'department', para ello se utiliza 'select fullname,department' para escoger las columnas a las cuales se les aplica la consulta y 'from employee' para determinar la tabla en cuestión, en este caso todo el archivo:

In [4]:
csvsql --query 'select fullname,maritalstatus from employee' employee > out_1
head out_1

fullname,maritalstatus
Fiona Ball,Common-Law
Palma Naylor,Married
Katharine Flemming,Common-Law
Shaun Colon,Single
Pearl Militello,Common-Law
Vern Largo,Single
Sabrina Vargas,Single
Sal Beaulieu,Single
Edna Vitale,Single


A través de `select distinct` se imprimen los registros únicos de la columna 'department':

In [5]:
csvsql --query 'select distinct department from employee' employee 

department
Accounting
Advertising
Asset Management
Customer Relations
Customer Service
Finances
Human Resources
Legal Department
Media Relations
Payroll
Public Relations
Quality Assurance
Sales and Marketing
Research and Development
Tech Support


El comando `limit` permite restringir el número de filas en cuestión. 

En el siguiente ejemplo se guardan los primeros 10 registros de las columnas `fullname` y `ssn`, y adicionalmente a través de `order by` se organiza alfabeticamente de acuerdo con los nombres:

In [6]:
csvsql --query 'select fullname,ssn from employee order by fullname limit 10' employee > out_2
cat out_2

fullname,ssn
Abby Mccray,496-99-2998
Abigail Harrell,242-97-5957
Albert Whitney,496-24-4704
Alexandria Beatty,242-97-4177
Andera Jolley,496-36-8473
Bridgett Bier,242-97-8961
Caridad Hutcherson,893-24-4950
Caridad Jenson,893-99-1977
Carla Polk,323-35-5441
Cary Hartnett,323-99-4177


**Ejercicio.--** Obtenga un archivo con los número de telefóno de los empleados casados por ciudad.

---

Las funciones `count` (conteo), `avg` (cálculo de media) y `sum` (suma) permiten realizar operaciones sobre filas y columnas en consulta.

A continuación se cuenta el número de empleados que pertenecen cada departamento a través de `select distinct`, agrupando por columnas con `group by` y nombrando la nueva columna con `as`. Adicionalmente se organiza de menor a mayor número de miembros a través de `order by`:

In [7]:
csvsql --query 'select distinct department,count(department) as total_empleados from employee group by department order by total_empleados' employee 

department,total_empleados
Advertising,1
Customer Relations,1
Finances,1
Legal Department,1
Quality Assurance,1
Tech Support,1
Accounting,2
Customer Service,2
Media Relations,2
Research and Development,2
Asset Management,3
Payroll,3
Human Resources,4
Public Relations,5
Sales and Marketing,21


Note que las funciones `order by` y `group by` se ubican al final del comando.


En el siguiente ejemplo se utiliza el archivo `order detail` para obtener la cantidad de artículos vendidos por producto y cuanto fue el total de la venta por artículo para dicha compra; esta columna se calcula multiplicando la cantidad por el precio:

In [8]:
csvsql --query 'select *,"quantity"*"price" as total from orderdetail group by "product.id"' orderdetail > out_3  
head out_3

n,order.id,product.id,customer.id,quantity,price,total
25925,5245,1,54,4,426,1704
25848,5227,2,20,9,583,5247
25821,5221,3,32,12,870,10440
25768,5211,4,7,5,831,4155
25857,5229,5,12,12,980,11760
25928,5246,6,119,13,643,8359
25945,5249,7,52,0,831,0
25808,5218,8,80,6,478,2868
25526,5159,9,73,12,891,10692


El simbolo  *  luego del comando `select` indica que se seleccionan todos los campos de la tabla.

Se desea conocer cual es el mínimo pedido dentro de cada orden, para ello es necesario agrupar los ingresos totales por cada pedido en cada orden y encontrar el mínimo entre ellos. 

In [9]:
csvsql --query 'select "order.id" as orden,min("quantity"*"price") as total from orderdetail group by orden' orderdetail > out_4
head out_4

orden,total
1,0
2,234
3,126
4,0
5,918
6,105
7,112
8,525
9,420


A continuación se requieren filtrar los pedidos que generan mas de 10.000 en ingresos.

In [10]:
csvsql --query 'select * from out_3 where total > 10000' out_3

n,order.id,product.id,customer.id,quantity,price,total
25821,5221,3,32,12,870,10440
25857,5229,5,12,12,980,11760
25526,5159,9,73,12,891,10692
25360,5123,16,45,13,806,10478
25750,5208,44,34,14,750,10500
25930,5246,54,10,11,976,10736
25932,5247,56,40,13,979,12727
25795,5215,65,19,11,926,10186
25564,5167,91,7,15,701,10515
25898,5238,99,115,15,707,10605
25866,5231,105,60,13,815,10595
25884,5235,118,24,15,683,10245
25892,5237,136,89,12,860,10320


Cabe resaltar que de acuerdo con los ingresos calculados, algunos campos son cero lo que denota que la orden fue gratis o por error se registró un pedido con cero unidades. Tomando este ejemplo, a continuación se filtran los campos defectuosos:

In [11]:
csvsql --query 'select * from out_3 where quantity = 0 or price = 0' out_3 > out_5
cat out_5

n,order.id,product.id,customer.id,quantity,price,total
25945,5249,7,52,0,831,0
25926,5245,12,50,0,852,0
25904,5241,18,108,0,418,0
25561,5166,48,5,0,741,0
25752,5209,81,125,0,640,0
25850,5228,133,15,0,675,0


**Ejercicio.--** Extraiga del archivo `order detail` los productos que generan mayor ingreso al negocio.

---

El comando `like` permite realizar consultas y filtros a través del patrón específico del registro.Para ello, cuenta con los siguientes operadores que funcionan como busquedas a través de expresiones regulares comunes:

    Like '%p'  <busca los campos que finalicen con p> 
    Like 'p%'  <busca los campos que inicien con p>
    Like '%p%' <busca los campos que contenga p>
    Like '_p%' <busca los campos que contengan una p en la segunda posición>

A continuación se filtran las personas pertenecientes a una misma familia a través del apellido en común:

In [14]:
csvsql --query 'select * from family where "fullname" like "%Bailey%"' family

num,parent.id,fullname,ssn,birthdate
1,19,Verlie Bailey,496-35-2171,2011-06-10 17:28:19.000000
24,19,Wilson Bailey,496-99-2998,2012-12-17 05:09:27.000000
27,19,Jean Bailey,661-35-4381,2014-07-24 18:09:03.000000
70,19,Bridgette Bailey,532-99-1147,2013-08-05 08:48:32.000000
114,19,Crista Bailey,125-35-5195,2012-11-13 10:11:57.000000
139,19,Barbra Bailey,496-24-2171,2013-04-26 04:51:34.000000


En el siguiente comando se seleccionan los nacimientos de hijos en 2016 organizados por fecha, mostrando el código del padre y el nombre completo:

In [15]:
csvsql --query 'select "parent.id","fullname", "birthdate" from family where "birthdate" like "2016%" order by "birthdate"' family

parent.id,fullname,birthdate
46,Contessa Tapp,2016-01-11 10:59:34.000000
38,Naomi Lafayette,2016-01-14 00:08:24.000000
34,Darryl Laster,2016-01-31 09:43:33.000000
38,Emmanuel Lafayette,2016-02-23 06:12:08.000000
35,Lula Petry,2016-03-08 16:04:28.000000
46,Arlene Tapp,2016-03-18 19:12:25.000000
24,Magdalena Renwick,2016-03-19 10:52:54.000000
25,Damaris Beebe,2016-05-01 10:18:08.000000
26,Lourdes Purvis,2016-05-24 05:55:03.000000
10,Bernarda Tippett,2016-05-29 11:47:28.000000
11,Hedwig Sawyers,2016-06-05 12:26:12.000000
9,Noah Huskey,2016-06-05 19:26:16.000000
1,Denny Durkin,2016-06-12 11:11:57.000000


En el siguiente comando se seleccionan los nacimientos de hijos en 2016 organizados por fecha, mostrando el código del padre y el nombre completo:

In [16]:
csvsql --query 'select company,count(company) as número from customer where "ccntype" like "%U.S.%" group by "company"' customer

company,número
A CUT ABOVE,1
AKRON ELEM SCHOOL,1
ALAMO HEIGHTS CENTER,1
APT FIRELANDS,1
BAYOU BEND TOWERS,1
BEACH PLACE ONE,1
BENEFITS CLAIMS INC,1
BRUNO ASSOCIATES,1
CLARKIN & TOWSEND,1
DORSI & SMITH,1
DR INGRID OSSWALD,1
DR MONGIA,1
ELIOT WOODS CONDO,1
ELMWOOD APTS OFFICE,1
EVANS DRYWELL,1
EXTRA BOXES,1
FARMINGDALE CONDOS,1
FRANK MONDAY,1
FRANS AUTO CENTER,1
FRAZIER APTS,1
G AND B CONTRUCTION,1


---

En el siguiente ejemplo se requiere guardar la información de 'product.key' y 'supplier.id' que se encuentra en el archivo `product`  para los productos registrados en el archivo anterior en donde reposa la cantidad y los ingresos.

In [17]:
csvsql --query 'select "product.key","supplier.id" from product left join out_3 on out_3."product.id"=product."product.id"' product out_3 > out_4
head out_4

product.key,supplier.id
YWAG895,4
CKZS960,1
BXXU863,8
AHJM307,8
ZOJB822,8
YFSM789,6
WAIZ521,1
VFAO391,2
TFWW446,9


**Ejercicio.--** Obtenga una tabla que contenga todos los archivos `order****`  agregando una columna que indique el año.

---

## Borrado de archivos temporales

A continuación se borran todos los archivos creados como ejemplos en este libro.

In [13]:
rm out*

---

SQL
===

**Juan David Velásquez Henao**  
jdvelasq@unal.edu.co   
Universidad Nacional de Colombia, Sede Medellín  
Facultad de Minas  
Medellín, Colombia

---

Haga click [aquí](https://github.com/jdvelasq/bash-for-analytics/tree/master/) para acceder al repositorio online.

Haga click [aquí](http://nbviewer.jupyter.org/github/jdvelasq/bash-for-analytics/tree/master/) para explorar el repositorio usando `nbviewer`. 